In [2]:
import requests
import json
import urllib.request
import pandas as pd
import ast

In [3]:
response_details_merge = pd.read_csv('../data/Data 6 - Merged Responses and Movie Details.csv')
response_details_merge.drop(columns='Unnamed: 0', inplace=True)
response_details_merge.head()

,id_x,person_mbti,mbti_test,character,movie,new_title,genre_ids,id_y,popularity,release_date,title,vote_average,vote_count
0,0,ENFP,michaelcaloz.com,Tony Stark,Avengers: Endgame,Avengers: Endgame,"[12, 878, 28]",299534,240.886,2019-04-24,Avengers: Endgame,8.255,24867
1,0,ENFP,michaelcaloz.com,Rocket Raccoon,Guardians of the Galaxy Vol. 3,Guardians of the Galaxy Vol. 3,"[878, 12, 28]",447365,169.896,2023-05-03,Guardians of the Galaxy Vol. 3,7.972,6262
2,0,ENFP,michaelcaloz.com,Captain America,Avengers: Endgame,Avengers: Endgame,"[12, 878, 28]",299534,240.886,2019-04-24,Avengers: Endgame,8.255,24867
3,0,ENFP,michaelcaloz.com,Peter Parker,Spider-Man: Far From Home,Spider-Man: Far From Home,"[28, 12, 878]",429617,124.720,2019-06-28,Spider-Man: Far From Home,7.400,15144
4,0,ENFP,michaelcaloz.com,Hobie Brown,Spider-Man: Across the Spider-Verse,Spider-Man: Across the Spider-Verse,"[16, 28, 12, 878]",569094,187.181,2023-05-31,Spider-Man: Across the Spider-Verse,8.366,6265
